In [1]:
import os
import sys
from tqdm import tqdm
from PIL import Image
import numpy as np

sys.path.extend(['..'])

from utils.config import process_config

import tensorflow as tf
from tensorflow.layers import (conv2d, max_pooling2d, average_pooling2d, batch_normalization, dropout, dense)
from tensorflow.nn import (relu, sigmoid, softmax, leaky_relu)

from sklearn.utils import shuffle

In [3]:
DATA = '../data/data_clean/'
CONF = '../configs/roman.json'

In [5]:
def normalize(image):
    return (image - image.min()) / (image.max() - image.min())

def shuffle_sim(a, b):
    assert a.shape[0] == a.shape[0], 'Shapes must be equal'
    
    ind = np.arange(a.shape[0])
    np.random.shuffle(ind)
    return a[ind], b[ind]

In [6]:
def read_train_test(path_to_data):
    data = {}
    for dset in ['train', 'test']:
        path_ = os.path.join(path_to_data, dset)
        X, Y = [], []
        classes = [d for d in os.listdir(path_) if os.path.isdir(os.path.join(path_, d))]
        classes.sort()
        
        for cl in classes:
            y = np.zeros((1, 8), dtype=np.int32)
            y[0, int(cl) - 1] = 1
            
            cl_path = os.path.join(path_, cl)
            filenames = [os.path.join(cl_path, pict) for pict in os.listdir(cl_path) if pict.endswith('.jpg')]
            
            for im in filenames:
                image = np.asarray(Image.open(im), dtype=np.float32)
                X.append(normalize(image).reshape((1, image.shape[0], image.shape[1], image.shape[2])))
                Y.append(y)
        
        a, b = shuffle_sim(np.concatenate(X), np.concatenate(Y))
        data[dset] = ([a, b])
    return data

In [19]:
class Model():
    
    def __init__(self, config, sess_cf, learning_rate):
        self.lr = learning_rate
        self.sess = tf.Session(config=sess_cf)

        self.x = tf.placeholder(dtype=tf.float32, shape=(None, config.image_size, config.image_size, 3))
        self.y = tf.placeholder(dtype=tf.int32, shape=(None, 8))
        self.training = tf.placeholder(dtype=tf.bool, shape=())

        global_step = tf.Variable(1, name='global_step', trainable=False, dtype=tf.int32)
        self.step = tf.assign(global_step, global_step + 1)
        
        self.model()
        
        self.summ_writer = tf.summary.FileWriter(config.summary_dir, graph=self.sess.graph)
        self.sess.run(tf.global_variables_initializer())
    
    def block(self, inp,
              ch,
              num,
              c_ker=[(3, 3), (3, 3)],
              c_str=[(1, 1), (1, 1)],
              act=relu,
              mp_ker=(2, 2),
              mp_str=(2, 2)):
    
        with tf.variable_scope('block_' + str(num), reuse=tf.AUTO_REUSE):
            conv = conv2d(inp, ch, c_ker[0], strides=c_str[0])
            bn = batch_normalization(conv)
            out = act(bn)
            out = dropout(out, 0.2)
            tf.summary.histogram('conv1', conv)
            print(out.shape)
            
            conv = conv2d(out, ch, c_ker[1], strides=c_str[1])
            bn = batch_normalization(conv)
            out = act(bn)
            tf.summary.histogram('conv2', conv)
            print(out.shape)
            
            out = max_pooling2d(out, mp_ker, strides=mp_str)
            print(out.shape)
        return out
    
    def model(self):
        with tf.name_scope('layers'):
            out = self.block(self.x, 32, 1, c_str=[(1, 1), (1, 1)])
            out = self.block(out, 64, 2, c_str=[(1, 1), (1, 1)])
            out = self.block(out, 128, 3, c_str=[(1, 1), (1, 1)])
            out = self.block(out, 256, 4, c_str=[(1, 1), (2, 2)])
            
            dim = np.prod(out.shape[1:])
            out = tf.reshape(out, [-1, dim])
            print(out.shape)
            
            dense_l = dense(out, 128)
            tf.summary.histogram('dense2', dense_l)
            out = batch_normalization(dense_l)
            out = leaky_relu(out, alpha=0.01)
            out = dropout(out, rate=0.7, training=self.training)
            print(out.shape)

            self.predictions = dense(out, 8, activation=softmax)
            tf.summary.histogram('pred', self.predictions)

        with tf.name_scope('metrics'):    
            amax_labels = tf.argmax(self.y, 1)
            amax_pred   = tf.argmax(self.predictions, 1)

            self.loss = tf.losses.softmax_cross_entropy(self.y, self.predictions)        
            self.acc = tf.reduce_mean(tf.cast(tf.equal(amax_labels, amax_pred), dtype=tf.float32))

            self.optimizer = tf.train.AdamOptimizer(self.lr).minimize(self.loss)

            tf.summary.scalar('loss', self.loss)
            tf.summary.scalar('accuracy', self.acc)

        self.summary = tf.summary.merge_all()
        
    def train(self, dat, dat_v, epochs):
        for epoch in range(epochs):
            loss, acc, _, summary, step = self.sess.run([
                self.loss, self.acc, self.optimizer, self.summary, self.step
            ],
                                                feed_dict={
                                                    self.x: dat[0],
                                                    self.y: dat[1],
                                                    self.training: True
                                                })

            self.summ_writer.add_summary(summary, step)
            print('EP: {:3d}\tLOSS: {:.10f}\tACC: {:.10f}'.format(
                epoch, loss, acc))

            if epoch % 10 == 0 and epoch != 0:
                self.test(dat_v)
                
    def test(self, dat):
        loss, acc = self.sess.run([self.loss, self.acc],
                                         feed_dict={self.x: dat[0],
                                                    self.y: dat[1],
                                                    self.training: False})

        print('\tVALIDATION\tLOSS: {:.10f}\tACC: {:.10f}'.format(loss, acc))
    
    def close(self):
        self.sess.close()
        tf.reset_default_graph()
        

In [20]:
m.close()

In [21]:
config_tf = tf.ConfigProto(allow_soft_placement=True)
config_tf.gpu_options.allow_growth = True
config_tf.gpu_options.per_process_gpu_memory_fraction = 0.95

In [22]:
config = process_config(CONF)
config['exp_name'] = '4b_mf_1'
config['summary_dir'] = '../experiments/' + config['exp_name'] + '/summary'

In [23]:
m = Model(config, config_tf, 5e-4)

(?, 126, 126, 32)
(?, 124, 124, 32)
(?, 62, 62, 32)
(?, 60, 60, 64)
(?, 58, 58, 64)
(?, 29, 29, 64)
(?, 27, 27, 128)
(?, 25, 25, 128)
(?, 12, 12, 128)
(?, 10, 10, 256)
(?, 4, 4, 256)
(?, 2, 2, 256)
(?, 1024)
(?, 128)


In [24]:
dat = read_train_test(DATA)

In [ ]:
m.train(dat['train'], dat['test'], 100)

EP:   0	LOSS: 1.3219293356	ACC: 0.9501084685
EP:   1	LOSS: 1.3271000385	ACC: 0.9479392767
EP:   2	LOSS: 1.3127039671	ACC: 0.9587852359
EP:   3	LOSS: 1.3346115351	ACC: 0.9414316416
EP:   4	LOSS: 1.3258922100	ACC: 0.9479392767
EP:   5	LOSS: 1.3181246519	ACC: 0.9587852359
EP:   6	LOSS: 1.3139684200	ACC: 0.9609544277
EP:   7	LOSS: 1.3143208027	ACC: 0.9631236196
EP:   8	LOSS: 1.3184775114	ACC: 0.9544468522
EP:   9	LOSS: 1.3273341656	ACC: 0.9436008930
EP:  10	LOSS: 1.3233304024	ACC: 0.9544468522
	VALIDATION	LOSS: 1.4170440435	ACC: 0.8666666746
EP:  11	LOSS: 1.3253732920	ACC: 0.9479392767
EP:  12	LOSS: 1.3310130835	ACC: 0.9457700849
EP:  13	LOSS: 1.3330909014	ACC: 0.9370932579
EP:  14	LOSS: 1.3196415901	ACC: 0.9544468522
EP:  15	LOSS: 1.3373776674	ACC: 0.9370932579
EP:  16	LOSS: 1.3284895420	ACC: 0.9436008930
EP:  17	LOSS: 1.3238655329	ACC: 0.9544468522
EP:  18	LOSS: 1.3218891621	ACC: 0.9501084685
EP:  19	LOSS: 1.3318897486	ACC: 0.9414316416
EP:  20	LOSS: 1.3308130503	ACC: 0.9414316416
	VALID